In [1]:
### Mixture density network to predict EURUSD currency.
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

In [38]:
# Read the excel file
eurusd = pd.read_excel('C:/Users/lenovo/Desktop/currency.xlsx',sheet_name = "EURUSD")
eurusd["price_t-1"] = eurusd['Last Price'].shift(1)
eurusd

,Date,Last Price,price_t-1
0,2022-06-27 22:00:00,1.0585,NaN
1,2022-06-27 22:01:00,1.0581,1.0585
2,2022-06-27 22:02:00,1.0584,1.0581
3,2022-06-27 22:03:00,1.0584,1.0584
4,2022-06-27 22:04:00,1.0584,1.0584
...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172
12848,2022-07-08 18:16:00,1.0171,1.0171
12849,2022-07-08 18:17:00,1.0172,1.0171
12850,2022-07-08 18:18:00,1.0170,1.0172


In [39]:
# Calculate the return at time t
eurusd['return_t'] = np.log(eurusd['Last Price']) - np.log(eurusd["price_t-1"])
eurusd

,Date,Last Price,price_t-1,return_t
0,2022-06-27 22:00:00,1.0585,NaN,NaN
1,2022-06-27 22:01:00,1.0581,1.0585,-0.000378
2,2022-06-27 22:02:00,1.0584,1.0581,0.000283
3,2022-06-27 22:03:00,1.0584,1.0584,0.000000
4,2022-06-27 22:04:00,1.0584,1.0584,0.000000
...,...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172,-0.000098
12848,2022-07-08 18:16:00,1.0171,1.0171,0.000000
12849,2022-07-08 18:17:00,1.0172,1.0171,0.000098
12850,2022-07-08 18:18:00,1.0170,1.0172,-0.000197


In [40]:
# Use data in the late 10 minutes as features
window = 10
for i in range(1,window + 1):
    string = 'return_t-' + str(i)
    eurusd[string] = eurusd['return_t'].shift(i)
eurusd

,Date,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10
0,2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197
12848,2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197
12849,2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000
12850,2022-07-08 18:18:00,1.0170,1.0172,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000


In [41]:
# Set data as index
eurusd.set_index('Date',inplace=True)
eurusd

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10
Date,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000


In [42]:
# Add S&P500 as feature
spx = pd.read_excel('C:/Users/lenovo/Desktop/indices.xlsx',sheet_name = "S&P500")
spx.set_index('Date',inplace=True)
spx.rename(columns={'Last Price':'SPX'},inplace=True)
spx = spx['SPX']
eurusd_spx = eurusd.merge(spx, how='outer', left_index=True, right_index=True)
eurusd_spx

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX
Date,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197,3889.08
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,3889.09
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,3888.80


In [43]:
# Add Euro Stoxx50 as feature
sx5e = pd.read_excel('C:/Users/lenovo/Desktop/indices.xlsx',sheet_name = "Euro Stoxx50")
sx5e.set_index('Date',inplace=True)
sx5e.rename(columns={'Last Price':'SX5E'},inplace=True)
sx5e = sx5e['SX5E']
eurusd_spx_sx5e = eurusd_spx.merge(sx5e, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E
Date,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197,3889.08,NaN
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,3889.09,NaN
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,3888.80,NaN


In [44]:
# Add implied volatility as feature
vol = pd.read_excel('C:/Users/lenovo/Desktop/implied_vol.xlsx',sheet_name = "EURUSDV1M")
vol.set_index('Date',inplace=True)
vol.rename(columns={'Last Price':'VOL'},inplace=True)
vol = vol['VOL']
eurusd_spx_sx5e_vol = eurusd_spx_sx5e.merge(vol, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e_vol

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL
Date,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625
2022-07-08 18:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625
2022-07-08 18:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625


In [45]:
# Add interest rate as feature

eusa10 = pd.read_excel('C:/Users/lenovo/Desktop/interest_rate.xlsx',sheet_name = "EUSA10")
eusa10.set_index('Date',inplace=True)
eusa10.rename(columns={'Last Price':'EUSA10'},inplace=True)
eusa10 = eusa10['EUSA10']
eurusd_spx_sx5e_vol_eusa10 = eurusd_spx_sx5e_vol.merge(eusa10, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e_vol_eusa10

ussw10 = pd.read_excel('C:/Users/lenovo/Desktop/interest_rate.xlsx',sheet_name = "USSW10")
ussw10.set_index('Date',inplace=True)
ussw10.rename(columns={'Last Price':'USSW10'},inplace=True)
ussw10 = ussw10['USSW10']
eurusd_spx_sx5e_vol_eusa10_ussw10 = eurusd_spx_sx5e_vol_eusa10.merge(ussw10, how='outer', left_index=True, right_index=True)
eurusd_com = eurusd_spx_sx5e_vol_eusa10_ussw10
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2755
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2760
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2747
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1863
2022-07-08 19:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1850
2022-07-08 19:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1862


In [46]:
# Fill NAN
eurusd_com = eurusd_com.fillna(method ='ffill')
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2755
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2760
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2747
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1863
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1850
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1862


In [47]:
# Find the first row that does not contain NAN. Delete all data before this row.

In [51]:
notna_index = eurusd_com[eurusd_com.notna().all(axis = 1)].index[0]
eurusd_com = eurusd_com.loc[notna_index : ,:]
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-28 14:30:00,1.0533,1.0530,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,-0.000095,0.000190,3923.17,3569.19,8.5650,2.4190,3.2972
2022-06-28 14:31:00,1.0530,1.0533,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,-0.000095,3925.45,3570.71,8.5650,2.4110,3.2910
2022-06-28 14:32:00,1.0530,1.0530,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,3937.14,3577.18,8.5750,2.4208,3.2975
2022-06-28 14:33:00,1.0528,1.0530,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,3940.04,3576.80,8.5750,2.4221,3.2979
2022-06-28 14:34:00,1.0531,1.0528,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,3940.33,3576.87,8.5750,2.4200,3.2903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,3887.84,3506.55,10.7625,2.1560,3.1863
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,3887.84,3506.55,10.7625,2.1560,3.1850
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,3887.84,3506.55,10.7625,2.1560,3.1862


In [52]:
# Split training set and testing set. The testing set accounts for 10% of the whole data.
num = eurusd_com.shape[0]//10
eurusd_train = eurusd_com.iloc[:-num,:]
eurusd_test = eurusd_com.iloc[-num: ,]

In [53]:
eurusd_train

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-28 14:30:00,1.0533,1.0530,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,-0.000095,0.000190,3923.17,3569.19,8.565,2.4190,3.2972
2022-06-28 14:31:00,1.0530,1.0533,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,-0.000095,3925.45,3570.71,8.565,2.4110,3.2910
2022-06-28 14:32:00,1.0530,1.0530,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,0.000190,3937.14,3577.18,8.575,2.4208,3.2975
2022-06-28 14:33:00,1.0528,1.0530,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,-0.000190,3940.04,3576.80,8.575,2.4221,3.2979
2022-06-28 14:34:00,1.0531,1.0528,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,0.000000,-0.000095,-0.000095,3940.33,3576.87,8.575,2.4200,3.2903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-07 23:17:00,1.0163,1.0163,0.000000,-0.000098,-0.000197,0.000000,0.000000,-0.000098,0.000000,0.000000,0.000098,0.000000,0.000000,3902.62,3488.50,10.710,2.1277,3.0830
2022-07-07 23:18:00,1.0163,1.0163,0.000000,0.000000,-0.000098,-0.000197,0.000000,0.000000,-0.000098,0.000000,0.000000,0.000098,0.000000,3902.62,3488.50,10.710,2.1277,3.0830
2022-07-07 23:19:00,1.0164,1.0163,0.000098,0.000000,0.000000,-0.000098,-0.000197,0.000000,0.000000,-0.000098,0.000000,0.000000,0.000098,3902.62,3488.50,10.710,2.1277,3.0830


In [54]:
# Nolmallize data by minmax scaler
from sklearn import preprocessing
# Construct a scaler
MiMaScaler = preprocessing.MinMaxScaler()
# Fit data on training set
MiMaScaler.fit(eurusd_train)
# Generate the scaled data
train = MiMaScaler.transform(eurusd_train)
scaled_eurusd_train = pd.DataFrame(train, columns=eurusd_train.columns,index = eurusd_train.index)
scaled_eurusd_train.describe()

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
count,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000,10738.000000
mean,0.579426,0.579510,0.374055,0.374083,0.373989,0.374020,0.374020,0.374017,0.374017,0.374050,0.374022,0.373987,0.374027,0.409203,0.445237,0.472969,0.464282,0.461863
std,0.305841,0.305820,0.044039,0.044072,0.044042,0.044033,0.044033,0.044034,0.044034,0.044043,0.044036,0.044045,0.044039,0.137854,0.226025,0.246500,0.259653,0.233668
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.288608,0.288608,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.387548,0.296246,0.278268,0.254772,0.293712
50%,0.716456,0.716456,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.419238,0.421201,0.415397,0.425658,0.410953
75%,0.787342,0.787342,0.399926,0.399926,0.399923,0.399923,0.399923,0.399923,0.399923,0.399926,0.399923,0.399923,0.399923,0.448784,0.580565,0.703288,0.634519,0.639402
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
# Scale testing set by using this scaler
test = MiMaScaler.transform(eurusd_test)
scaled_eurusd_test = pd.DataFrame(test, columns=eurusd_test.columns,index = eurusd_test.index)
scaled_eurusd_test.describe()

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
count,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000,1193.000000
mean,0.024580,0.024563,0.375060,0.372669,0.376245,0.375060,0.373876,0.375082,0.376245,0.374996,0.374996,0.377365,0.376116,0.787638,0.586275,0.738837,0.361197,0.574461
std,0.057449,0.057441,0.062228,0.063165,0.062449,0.062219,0.062455,0.062214,0.062449,0.062232,0.062232,0.063156,0.062451,0.032158,0.048964,0.039015,0.039652,0.091497
min,-0.169620,-0.169620,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,0.634538,0.430875,0.689655,0.272051,0.469371
25%,-0.002532,-0.002532,0.349165,0.349122,0.349165,0.349165,0.349165,0.349165,0.349165,0.349165,0.349165,0.349165,0.349165,0.796061,0.580565,0.708099,0.329114,0.506694
50%,0.040506,0.040506,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.796061,0.580565,0.720930,0.370705,0.529817
75%,0.063291,0.063291,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400662,0.400624,0.796061,0.610027,0.773857,0.370705,0.665720
max,0.116456,0.116456,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.872507,0.679240,0.842823,0.443239,0.755781


In [56]:
# Split label and features
# For training set
train_label = scaled_eurusd_train[['return_t']]
train_label

,return_t
Date,
2022-06-28 14:30:00,0.449346
2022-06-28 14:31:00,0.300430
2022-06-28 14:32:00,0.374888
2022-06-28 14:33:00,0.325238
2022-06-28 14:34:00,0.449360
...,...
2022-07-07 23:17:00,0.374888
2022-07-07 23:18:00,0.374888
2022-07-07 23:19:00,0.400606


In [57]:
train_feature = scaled_eurusd_train.loc[:,'return_t-1':]
train_feature

,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,
2022-06-28 14:30:00,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,0.325266,0.424510,0.350076,0.424515,0.896251,0.936970,0.001604,0.955998,0.970791
2022-06-28 14:31:00,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,0.325266,0.424510,0.350076,0.907367,0.943684,0.001604,0.939924,0.958215
2022-06-28 14:32:00,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,0.325266,0.424510,0.964361,0.972261,0.004812,0.959614,0.971400
2022-06-28 14:33:00,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,0.325266,0.978499,0.970583,0.004812,0.962226,0.972211
2022-06-28 14:34:00,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,0.979913,0.970892,0.004812,0.958007,0.956795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-07 23:17:00,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,0.400598,0.374888,0.374888,0.796061,0.580565,0.689655,0.370705,0.536308
2022-07-07 23:18:00,0.374888,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,0.400598,0.374888,0.796061,0.580565,0.689655,0.370705,0.536308
2022-07-07 23:19:00,0.374888,0.374888,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,0.400598,0.796061,0.580565,0.689655,0.370705,0.536308


In [58]:
# For testing set
test_label = scaled_eurusd_test[['return_t']]
test_feature = scaled_eurusd_test.loc[:,'return_t-1':]

In [59]:
# Build the model
event_shape = [1]
num_components = 5  # normal mixture with 5 components
params_size = tfp.layers.MixtureNormal.params_size(num_components, event_shape)
model = tfk.Sequential([
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(12, activation='relu'),
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(params_size, activation=None),
    tfpl.MixtureNormal(num_components, event_shape)
])

In [60]:
# Fit
model.compile(optimizer="adam",
              loss=lambda y, model: -model.log_prob(y))
model.fit(train_feature, train_label, batch_size=64, epochs=300)

Epoch 1/300
168/168 [==============================] - 1s 1ms/step - loss: -0.3646
Epoch 2/300
168/168 [==============================] - 0s 1ms/step - loss: -1.6905
Epoch 3/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7206
Epoch 4/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7364
Epoch 5/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7448
Epoch 6/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7509
Epoch 7/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7549
Epoch 8/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7514
Epoch 9/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7618
Epoch 10/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7703
Epoch 11/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7725
Epoch 12/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7712
E

168/168 [==============================] - 0s 1ms/step - loss: -1.8268
Epoch 99/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8319
Epoch 100/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8291
Epoch 101/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8323
Epoch 102/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8325
Epoch 103/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8261
Epoch 104/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8311
Epoch 105/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8297
Epoch 106/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8327
Epoch 107/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8317
Epoch 108/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8359
Epoch 109/300
168/168 [==============================] - 0s 1ms/step - loss: -1.

168/168 [==============================] - 0s 1ms/step - loss: -1.8446
Epoch 195/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8470
Epoch 196/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8529
Epoch 197/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8537
Epoch 198/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8516
Epoch 199/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8530
Epoch 200/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8502
Epoch 201/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8498
Epoch 202/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8488
Epoch 203/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8528
Epoch 204/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8517
Epoch 205/300
168/168 [==============================] - 0s 1ms/step - loss: -1

168/168 [==============================] - 0s 1ms/step - loss: -1.8652
Epoch 291/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8843
Epoch 292/300
168/168 [==============================] - 0s 1ms/step - loss: -1.9393
Epoch 293/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8537
Epoch 294/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8185
Epoch 295/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8289
Epoch 296/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8429
Epoch 297/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8499
Epoch 298/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8531
Epoch 299/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8494
Epoch 300/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8534


In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                160       
                                                                 
 dense_1 (Dense)             (None, 12)                132       
                                                                 
 dense_2 (Dense)             (None, 10)                130       
                                                                 
 dense_3 (Dense)             (None, 15)                165       
                                                                 
 mixture_normal (MixtureNorm  ((None, 1),              0         
 al)                          (None, 1))                         
                                                                 
Total params: 587
Trainable params: 587
Non-trainable params: 0
__________________________________________________________

In [62]:
# Create two empty columns
test_label["mix_mean"] = np.nan
test_label["mix_dev"] = np.nan
test_label

<ipython-input-62-1f5e28c17cee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label["mix_mean"] = np.nan
<ipython-input-62-1f5e28c17cee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label["mix_dev"] = np.nan


,return_t,mix_mean,mix_dev
Date,,,
2022-07-07 23:22:00,0.400608,NaN,NaN
2022-07-07 23:23:00,0.349167,NaN,NaN
2022-07-07 23:24:00,0.374888,NaN,NaN
2022-07-07 23:25:00,0.374888,NaN,NaN
2022-07-07 23:26:00,0.349165,NaN,NaN
...,...,...,...
2022-07-08 19:10:00,0.374888,NaN,NaN
2022-07-08 19:11:00,0.374888,NaN,NaN
2022-07-08 19:12:00,0.374888,NaN,NaN


In [63]:
for i in range(test_feature.shape[0]):
    x = np.array(test_feature.iloc[i])
    x = x.reshape(1,x.shape[0])
    gm = model(x)
    gm_weights = gm.mixture_distribution.probs_parameter().numpy().flatten()
    gm_means = gm.components_distribution.mean().numpy().flatten()
    gm_vars = gm.components_distribution.variance().numpy().flatten()
    mix_mean = np.dot(gm_weights, gm_means)
    mix_dev = np.sqrt(np.dot(gm_weights**2, gm_vars))
    test_label.iloc[i,1] = mix_mean
    test_label.iloc[i,2] = mix_dev

D:\anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-63-dc538d6fefb8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label.iloc[i,1] = mix_mean
<ipython-input-63-dc538d6fefb8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label.iloc[i,2] = mix_dev


In [64]:
test_label

,return_t,mix_mean,mix_dev
Date,,,
2022-07-07 23:22:00,0.400608,0.378637,0.021610
2022-07-07 23:23:00,0.349167,0.377480,0.023502
2022-07-07 23:24:00,0.374888,0.378082,0.022837
2022-07-07 23:25:00,0.374888,0.376526,0.022521
2022-07-07 23:26:00,0.349165,0.377722,0.022292
...,...,...,...
2022-07-08 19:10:00,0.374888,0.380714,0.023728
2022-07-08 19:11:00,0.374888,0.380676,0.023706
2022-07-08 19:12:00,0.374888,0.380711,0.023727


In [65]:
test_label['a'] = test_label['mix_mean'] - 1.96 * test_label['mix_dev']
test_label['b'] = test_label['mix_mean'] + 1.96 * test_label['mix_dev']

<ipython-input-65-f2e50151ce47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label['a'] = test_label['mix_mean'] - 1.96 * test_label['mix_dev']
<ipython-input-65-f2e50151ce47>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label['b'] = test_label['mix_mean'] + 1.96 * test_label['mix_dev']


In [66]:
test_label

,return_t,mix_mean,mix_dev,a,b
Date,,,,,
2022-07-07 23:22:00,0.400608,0.378637,0.021610,0.336281,0.420993
2022-07-07 23:23:00,0.349167,0.377480,0.023502,0.331417,0.423543
2022-07-07 23:24:00,0.374888,0.378082,0.022837,0.333322,0.422842
2022-07-07 23:25:00,0.374888,0.376526,0.022521,0.332385,0.420666
2022-07-07 23:26:00,0.349165,0.377722,0.022292,0.334030,0.421414
...,...,...,...,...,...
2022-07-08 19:10:00,0.374888,0.380714,0.023728,0.334206,0.427222
2022-07-08 19:11:00,0.374888,0.380676,0.023706,0.334211,0.427141
2022-07-08 19:12:00,0.374888,0.380711,0.023727,0.334207,0.427216


In [70]:
# See if the return_t falls in the confidence interval
result = (test_label['return_t'] > test_label['a']) & (test_label['return_t'] < test_label['b'])
result.value_counts()

True     892
False    301
dtype: int64

In [71]:
# Calculate the accuracy
accuracy = (result.value_counts().loc[True])/(test_label.shape[0])
accuracy

0.7476948868398994